# LangChain Chains

In LangChain, chains are modular sequences of components that automate and structure interactions with language models (LLMs). A chain defines the flow of data—from input processing and prompt construction to LLM execution and output parsing—allowing developers to build sophisticated LLM-driven applications with clarity and reusability.

Rather than calling an LLM directly with a prompt, chains help encapsulate logic, dependencies, and additional features like memory, tools, or post-processing.

Types of Chains in LangChain

- Simple Chains (LLMChain)
- Sequential Chains
- Router Chains
- MultiPromptChain
- Custom Chains

# Setting Up Work Environment

In [ ]:
!pip install --upgrade google-generativeai

In [ ]:
!pip install -q -U google-genai

In [ ]:
!pip install langchain-community

In [ ]:
!pip install -U langchain-google-genai

In [42]:
import os
from google import genai
import google.generativeai as ggenai
from google.colab import userdata
from IPython.display import display
from IPython.display import Markdown
import pandas as pd

from PIL import Image
from google.genai import types

from IPython.display import HTML

In [52]:
from langchain_google_genai import ChatGoogleGenerativeAI

from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

from langchain.chains import SimpleSequentialChain
from langchain.chains import SequentialChain

from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [7]:
# Set up the API key (Replace 'YOUR_API_KEY' with your actual Gemini API key)
key = userdata.get('genai_api')
client = genai.Client(api_key=key)

List the set of available models

In [9]:
ggenai.configure(api_key=key)

models = ggenai.list_models()
# for model in models:
#     print(model.name)

In [11]:
# Initialize Gemini LLM
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",       # or "gemini-1.5-pro", etc.
    temperature=0.9,
    google_api_key=key
)

# Simple Chains (LLMChain)

The LLMChain is the simplest and most fundamental type of chain in LangChain. It is designed to streamline the process of sending inputs through a prompt template to a large language model (LLM) and returning the result. This structure is ideal for use cases where a single prompt is sufficient to generate a response.

- We will now initialize a prompt that accepts a variable named product. This prompt will instruct the language model to generate the most suitable name for a company that produces the specified product.

In [13]:
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

In [14]:
chain = LLMChain(llm=llm, prompt=prompt)

/tmp/ipython-input-14-1305865249.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt)


In [17]:
product = "Queen Size Sheet Set"

In [23]:
chain.invoke({"product": product})

{'product': 'Queen Size Sheet Set',
 'text': 'The best name will depend on your brand\'s desired image and target audience.  Here are some options categorized by approach:\n\n**Elegant & Upscale:**\n\n* Royal Rest\n* Empress Linens\n* Sovereign Sheets\n* Regal Sleep\n* Velvet Slumber\n* The Queen\'s Collection\n\n**Modern & Minimalist:**\n\n* Queen Sheets\n* Slumber Co.\n* Bed & Thread\n* Linen & Loom\n* The Sheet Society\n* Restful Nights\n\n**Cozy & Comfortable:**\n\n* Cozy Queen\n* Sweet Dreams Sheets\n* Cloud Nine Linens\n* Slumberland Sheets\n* The Comfort Collection\n* Peaceful Nights\n\n\n**Creative & Playful:**\n\n* Queen\'s Quarters\n* Sheet Happens\n* The Big Sleep\n* Dream Weaver Linens\n\n\n**Tips for Choosing:**\n\n* **Check for availability:** Make sure the name isn\'t already taken (website domain, trademark).\n* **Keep it short and memorable:**  Easy to recall and share.\n* **Reflect your brand:** Does it align with your values and target audience?\n* **Say it out loud:

In [22]:
chain.invoke({"product": product})['text']

"The best name depends on the desired brand image. Here are some options, categorized by the image they project:\n\n**Luxury & High-End:**\n\n* Royal Rest\n* Empress Linens\n* Sovereign Sheets\n* Regal Slumber\n* The Queen's Chamber\n\n**Modern & Minimalist:**\n\n* Queen Set\n* Slumber & Co.\n* Linen & Thread\n* The Sheet Co.\n* Restful Nights\n\n**Cozy & Comfortable:**\n\n* Cozy Queen\n* Sweet Dreams Sheets\n* Dream Weaver Linens\n* The Slumber Nook\n* Cloud Nine Bedding\n\n**Playful & Unique:**\n\n* Queen Bee Sheets\n* The Queen's Bed\n* Sheet Happens\n* Zzz's & Co.\n* Slumber Party Linens\n\n\n**Things to consider when choosing:**\n\n* **Target audience:** Who are you trying to reach?  A younger audience might appreciate a playful name, while an older audience might prefer something more sophisticated.\n* **Brand personality:**  Do you want to be seen as luxurious, affordable, playful, or sophisticated?\n* **Availability:** Check if the name is available as a website domain and trad

# Sequential Chains

SequentialChain is an advanced chain type in LangChain that allows multiple chains to be executed in a defined sequence, while managing multiple input and output variables across steps. Unlike SimpleSequentialChain, which only handles a single string input and output, SequentialChain supports complex workflows where each step can depend on multiple named variables and produce multiple outputs.

## Simple Sequential Chains

SimpleSequentialChain is a type of chain in LangChain that allows you to execute multiple LLM chains in a fixed, linear order. The output of one chain is automatically passed as the input to the next. This structure is ideal for straightforward, step-by-step workflows where each step builds directly on the result of the previous one.

- We will begin by creating the first chain, which consists of a language model and a prompt. This prompt will accept a product as input and generate an appropriate name for a company that produces that product.

- Next, we will define a second chain. This chain will take the generated company name as input and produce a concise, 20-word description of the company.

In [25]:
# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [31]:
# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company:{company_name}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [32]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two], verbose=True)

In [33]:
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
The best name will depend on your brand's desired image and target audience. Here are some options, categorized by approach:

**Luxury & High-End:**

* Royal Rest
* Empress Linens
* Sovereign Sheets
* The Queen's Chamber
* Serene Slumber (implies luxury and comfort)
* Celestial Sleep

**Simple & Direct:**

* Queen Size Sheets
* Queen Sheets Direct
* The Queen's Set
* Simply Queen
* Cozy Queen

**Modern & Chic:**

* Slumber & Style
* Thread & Bloom
* Linen & Luxe
* The Sheet Society
* Dream Weaver Linens

**Playful & Approachable:**

* Queen Bee Sheets
* Sleepy Queen
* Night Owl Linens
* The Comfy Queen
* Sweet Dreams Sheets

**Focusing on a specific material:**

* [Material] Queen Sheets (e.g., Egyptian Cotton Queen Sheets, Bamboo Queen Sheets)


**Tips for choosing the best name:**

* **Check for availability:** Make sure the name isn't already in use and that the domain name is available.
* **Keep it short and memorable:** Easy to recal

'Luxury bedding for discerning sleepers.  Elegant designs, superior comfort, exceptional quality.'

## Complex Sequential Chain (SequentialChain)

The SequentialChain in LangChain—often referred to as a complex sequential chain—is a powerful and flexible tool for executing multiple language model (LLM) steps in a defined sequence. Unlike SimpleSequentialChain, which only handles a single input and output, SequentialChain supports multiple named input and output variables, allowing for rich, multi-step workflows.

In [43]:
df = pd.read_csv('Data.csv')
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


We will be using the previously defined data, which contains a customer review.

- `First Chain`: This step will translate the original review into English.

- `Second Chain`: Using the translated English text from the first chain, this step will generate a one-sentence summary of the review.

- `Third Chain`: This step will identify the original language of the review. It operates directly on the initial review input.

- `Fourth Chain`: The final step takes multiple inputs—the summary produced by the second chain and the language identified in the third chain—and generates an appropriate follow-up response written in the detected language.

This sequential flow ensures that each chain builds on the outputs of previous steps, enabling a cohesive, multi-stage processing pipeline.

In [36]:
# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{Review}"
)
# chain 1: input= Review and output= English_Review
chain_one = LLMChain(llm=llm, prompt=first_prompt, output_key="English_Review")

In [37]:
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)
# chain 2: input= English_Review and output= summary
chain_two = LLMChain(llm=llm, prompt=second_prompt, output_key="summary")

In [38]:
# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt, output_key="language")

In [39]:
# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt, output_key="followup_message")

In [40]:
# overall_chain: input= Review
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary","followup_message"],
    verbose=True
)

In [44]:
review = df.Review[5]
overall_chain(review)

/tmp/ipython-input-44-1992003631.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  overall_chain(review)




> Entering new SequentialChain chain...

> Finished chain.


{'Review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
 'English_Review': "I find the taste mediocre. The head doesn't hold, it's strange. I buy the same ones in stores and the taste is much better...\n\nOld batch or counterfeit!?",
 'summary': "The reviewer found the beer's taste inferior to others they've had, suspecting it may be old or counterfeit.",
 'followup_message': "Plusieurs options sont possibles, dépendant du ton souhaité :\n\n**Option 1 (Formel et poli) :**\n\n> Nous vous remercions pour votre commentaire.  Nous regrettons que la dégustation de notre bière ne vous ait pas pleinement satisfait.  Nous prenons vos soupçons concernant la fraîcheur ou l'authenticité du produit très au sérieux et allons mener une enquête interne.  Pourriez-vous nous fournir plus de détails, tels que le numéro de lot et le lieu d'achat ?\n\n**Option 2 (Plus informel et engag

# Router Chain

A Router Chain in LangChain is a specialized chain that dynamically routes user input to one of several predefined sub-chains based on the content or intent of the input. This allows developers to build intelligent, multi-skill systems that can select the appropriate response logic depending on the user’s needs.

Up to this point, we have explored the use of LLMChain and SequentialChain. However, more complex scenarios often require additional logic. A common and practical pattern is routing input dynamically to different chains based on the nature or intent of that input.

This can be effectively achieved using a Router Chain, which acts as a decision-making layer. It evaluates the input and determines which specialized subchain should handle the request. Each subchain is designed to process a specific category or type of input.

For instance, consider a system that needs to route inputs based on the subject matter. You could define several prompt templates—each tailored to a specific domain. For example:

- One prompt optimized for answering physics questions,

- Another designed for math queries,

- A third for history topics,

- And a fourth for computer science.

The router analyzes the input, determines the relevant subject area, and directs the input to the appropriate subchain using the corresponding prompt. Let's proceed by defining the prompt templates for each subject area.

In [45]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts,
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity.

Here is a question:
{input}"""

- Once the prompt templates are defined, we can enhance them by assigning each one a name and a description. These metadata elements provide context about the purpose and specialization of each subchain. This information is then passed to the Router Chain, enabling it to make informed decisions about which subchain to route a given input to, based on the input’s content and intent.

In [46]:
prompt_infos = [
    {
        "name": "physics",
        "description": "Good for answering questions about physics",
        "prompt_template": physics_template
    },
    {
        "name": "math",
        "description": "Good for answering math questions",
        "prompt_template": math_template
    },
    {
        "name": "History",
        "description": "Good for answering history questions",
        "prompt_template": history_template
    },
    {
        "name": "computer science",
        "description": "Good for answering computer science questions",
        "prompt_template": computerscience_template
    }
]

In this scenario, we require a MultiPromptChain, which is specifically designed for routing between multiple prompt templates. As demonstrated, each routing option corresponds to a distinct prompt template. However, it’s important to note that routing is not limited to prompt templates alone—you can route between any type of chain, including those with additional logic or structure.

To implement this, we will also introduce two additional components:

- `LLMRouterChain`: This chain uses a language model to analyze the input and determine which subchain is most appropriate. The names and descriptions defined earlier for each subchain are used here to guide the routing decision.

- `RouterOutputParser`: This utility parses the LLM's output into a structured dictionary format. It identifies the selected destination chain and extracts any relevant input variables, making the data suitable for downstream processing.

With the necessary components in place, we can proceed to define and configure the language model that will be used for routing decisions. Let’s begin by importing and initializing the model.

In [48]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain

destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

As illustrated, each destination chain is implemented as an LLMChain, utilizing a language model with a specific prompt tailored to a particular subject area. In addition to these destination chains, a default chain must also be defined. This default chain serves as a fallback and is invoked when the Router Chain is unable to confidently determine which subchain is appropriate for the given input.

For example, in the context of our earlier use case—where subchains handle physics, math, history, and computer science—the default chain would be triggered if the input question does not align with any of those predefined categories. This ensures graceful handling of unexpected or ambiguous inputs.

In [49]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

- Next, we define the routing prompt template that will be used by the language model to determine which subchain to invoke. This template includes clear instructions on how the model should interpret the input and select the appropriate task, along with the expected output format. The output must follow a specific structure so that it can be accurately parsed by the RouterOutputParser and routed to the correct destination chain with the appropriate input.

In [50]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

- Next, we construct a PromptTemplate using the routing instructions defined earlier. This prompt is then used to create the Router Chain by passing it, along with the configured language model, into the LLMRouterChain constructor. This setup enables the model to analyze inputs and determine the correct subchain based on the routing logic.

In [53]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

It is important to highlight the inclusion of the Router Output Parser in this setup. This component plays a crucial role by parsing the language model’s response, allowing the router chain to determine which subchain should handle the input.

With all components defined, we can now assemble the overall multi-route chain. This final structure includes:

- `The Router Chain`, responsible for evaluating input and determining the routing path.

- `The Destination Chains`, each tailored to a specific task or subject.

- `The Default Chain`, which handles any input that does not match the criteria for the defined destination chains.

This configuration enables dynamic, intent-driven routing across multiple specialized processing chains.

In [54]:
chain = MultiPromptChain(router_chain=router_chain,
                         destination_chains=destination_chains,
                         default_chain=default_chain, verbose=True
                        )

/tmp/ipython-input-54-3038952769.py:1: LangChainDeprecationWarning: Please see migration guide here for recommended implementation: https://python.langchain.com/docs/versions/migrating_chains/multi_prompt_chain/
  chain = MultiPromptChain(router_chain=router_chain,


In [56]:
chain.run("What is Machine Learning?")



> Entering new MultiPromptChain chain...
computer science: {'input': 'What is Machine Learning?'}
> Finished chain.


"Machine learning (ML) is a branch of artificial intelligence (AI) and computer science which focuses on the use of data and algorithms to imitate the way that humans learn, gradually improving its accuracy.  Instead of being explicitly programmed to perform a task, a machine learning system learns from data.  This learning process involves identifying patterns, making predictions, and improving its performance over time without explicit human intervention for each new input.\n\nHere's a breakdown in imperative steps, illustrating the core process:\n\n1. **Data Acquisition and Preparation:** Gather relevant data. This might involve collecting data from various sources, cleaning it (handling missing values, outliers, etc.), and transforming it into a suitable format for the chosen algorithm (e.g., normalization, feature engineering).\n\n2. **Algorithm Selection:** Choose a machine learning algorithm appropriate for the task.  This depends on several factors including the type of data (s

In [57]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


'The question is: What is 2 + 2?\n\nThis is a simple addition problem.\n\n**Component Part 1:**  The number 2 represents two units.\n\n**Component Part 2:** The "+" symbol indicates addition, meaning we combine the units.\n\n**Component Part 3:** Combining two units and two more units results in a total of four units.\n\n**Answer:** Therefore, 2 + 2 = 4'

In [58]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


"Black body radiation is the electromagnetic radiation emitted by an idealized object, called a black body, that perfectly absorbs all incoming radiation at all wavelengths.  It's characterized by a specific spectrum that depends only on the object's temperature.  The hotter the object, the more intensely it radiates, and the shorter the wavelengths at which it emits most strongly."

In [59]:
chain.run("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...
None: {'input': 'Why does every cell in our body contain DNA?'}
> Finished chain.


"Every cell in our body contains DNA because DNA holds the instructions for building and maintaining the entire organism.  It's the blueprint for life.  These instructions are crucial for:\n\n* **Protein synthesis:** DNA contains the genes that code for the production of proteins. Proteins are the workhorses of the cell, carrying out a vast array of functions, from structural support to enzymatic activity.  Every cell needs to produce proteins to function.\n\n* **Cell division and replication:** When a cell divides, it needs to pass on a complete copy of its DNA to each daughter cell. This ensures that each new cell has the necessary instructions to function correctly.\n\n* **Cellular regulation and maintenance:** DNA plays a role in regulating gene expression, determining which proteins are produced and when. This is vital for maintaining the cell's health and function, and for responding to changes in the environment.\n\n* **Inheritance:** The DNA we inherit from our parents determin

In [60]:
chain.run("When was The World War I?")



> Entering new MultiPromptChain chain...
History: {'input': 'When was World War I?'}
> Finished chain.


"World War I took place from **1914 to 1918**.  More specifically, the war began on 28 July 1914 with Austria-Hungary's declaration of war on Serbia, following the assassination of Archduke Franz Ferdinand, and concluded with the Armistice of 11 November 1918.  While the fighting largely ceased on that date, the formal peace treaties that ended the state of war were not signed until the following year."